In [1]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
import seaborn as sns
import numpy as np
import pandas as pd


train_raw=pd.read_csv('train.csv')
target_data=train_raw['y']
data_train=train_raw.drop('y', axis = 1)


data_test=pd.read_csv('test.csv')
combine=[data_train,data_test]
train = pd.concat(combine)

In [2]:
#Target_data is skewed,usinglog to nomalize it
#target_data=np.log(target_data)

# Wrangle Data

In [3]:
raw_data=train.drop('ID', axis = 1)

Converting a categorical feature with labelEncoder

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

categorical=list(raw_data.select_dtypes(include=['object']).columns.values)
print(categorical)
for value in categorical:
    le.fit(raw_data[value])
    raw_data[value]=le.transform(raw_data[value])
raw_data.head()

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,37,23,20,0,3,27,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,37,21,22,4,3,31,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,24,24,38,2,3,30,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,24,21,38,5,3,30,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24,23,38,5,3,14,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


Shuffle and Split Data

In [5]:
from sklearn.cross_validation import train_test_split
feature_train_all=raw_data[:4209]
feature_test=raw_data[4209:]
X_train, X_test, y_train, y_test = train_test_split(feature_train_all,target_data, test_size = 0.2, random_state = 0)


/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Modeling

1. Looks like Ridge wins,we use Ridge to build a benchmark

In [20]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
from sklearn import linear_model
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    
    regressor = linear_model.Ridge(random_state=0)

   
    params = {'alpha':[105,106,107,108,109,110,111,112,113,114],'max_iter':[1,2,3,4,5,6,7,8,9]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [21]:
model=fit_model(X_train, y_train)

In [22]:
print(model.get_params()['alpha'])
print(model.get_params()['max_iter'])

105
1


In [18]:
reg=linear_model.Ridge(alpha=105,max_iter=10)
reg.fit(X_train, y_train)

Ridge(alpha=105, copy_X=True, fit_intercept=True, max_iter=10,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [19]:
y_predict=reg.predict(X_test)
print(r2_score( y_test,y_predict))

0.444377071101


# Your submission scored 0.54305, which is an improvement ofyour previous score of 0.54295

2.RandomForest

In [8]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor =  RandomForestRegressor(random_state=0)

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'n_estimators':[10,30,50,70,100,130,150,170,200]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [22]:
model=fit_model(feature_train_all, target_data)

KeyboardInterrupt: 

In [10]:
print(model.get_params()['n_estimators'])

150


In [30]:
regressor =  RandomForestRegressor(n_estimators=150)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=150, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [31]:
y_predict=regressor.predict(X_test)

In [32]:
print(r2_score( y_test,y_predict))

0.41880594082


3.Lasso

In [74]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
from sklearn import linear_model
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor = linear_model.Lasso(random_state=0)

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'alpha':[0.00001,0.00003,0.00006,0.00009,0.0001,0.0003,0.0006,0.0009,0.001,0.003,0.006,0.009,0.01,0.1,1,5,10,15,20,25,30]}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [75]:
model=fit_model(feature_train_all, target_data)

In [76]:
print(model.get_params()['alpha'])

0.0001


In [38]:
regressor =  linear_model.Lasso(alpha=0.0001)
regressor.fit(X_train, y_train)

/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
y_predict=regressor.predict(X_test)
print(r2_score( y_test,y_predict))

0.424769402121


4.ElasticNet

In [41]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
from sklearn.linear_model import ElasticNet
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor = ElasticNet(random_state=0)

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    #params = {'l1_ratio':[0.1,0.3,0.6,0.9,1.0,1.3,1.6,1.9,2.0,2.3,2.6,2.9],'alpha':[0.00001,0.00003,0.00006,0.00009,0.0001,0.0003,0.0006,0.0009,0.001,0.003,0.006,0.009,0.01]}
    params = {'l1_ratio':np.arange(0.0001,0.01,0.0001),'alpha':np.arange(0.01,1,0.01)}
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [ ]:
model=fit_model(feature_train_all, target_data)

In [80]:
print(model.get_params()['alpha'])
print(model.get_params()['l1_ratio'])

0.001
0.1


In [42]:
regressor =  ElasticNet(alpha=0.001,l1_ratio=0.1)
regressor.fit(X_train, y_train)

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [43]:
y_predict=regressor.predict(X_test)
print(r2_score( y_test,y_predict))

0.449834459337


5.svm.SVR(kernel='linear')

In [46]:
from sklearn.metrics import make_scorer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor = svm.SVR(kernel='linear')

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'C':[0.1,0.3,0.6,0.9,1.0,1.3,1.6,1.9,2.0]}
    #params = {'l1_ratio':np.arange(0.0001,0.01,0.0001),'alpha':np.arange(0.01,1,0.01)}
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [47]:
model=fit_model(X_train, y_train)

In [48]:
print(model.get_params()['C'])

0.3


In [49]:
y_predict=model.predict(X_test)
print(r2_score( y_test,y_predict))

0.425898270408


6.svm.SVR(kernel='rbf')

In [51]:
from sklearn.metrics import make_scorer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor = svm.SVR(kernel='rbf')


    params = {'C':[0.1,0.3,0.6,0.9,1.0,1.3,1.6,1.9,2.0],'epsilon':[0.01,0.03,0.06,0.09,0.1,0.3,0.6,0.9,1.0]}
    
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [52]:
model=fit_model(X_train, y_train)

In [53]:
print(model.get_params()['C'])
print(model.get_params()['epsilon'])

2.0
1.0


In [54]:
y_predict=model.predict(X_test)
print(r2_score( y_test,y_predict))

0.328778071876


7.AdaBoostRegressor

In [55]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    # TODO: Create a decision tree regressor object
    regressor = AdaBoostRegressor(random_state=1)


    params = {'n_estimators':[30,50,70,100,130,150],'learning_rate':[0.1,0.3,0.6,0.9,1.0,1.3,1.6,2.0]}
    
    
    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [56]:
model=fit_model(X_train, y_train)

In [57]:
print(model.get_params()['n_estimators'])
print(model.get_params()['learning_rate'])

30
0.1


In [58]:
y_predict=model.predict(X_test)
print(r2_score( y_test,y_predict))

0.449811977616


8.LassoLarsCV

In [60]:
from sklearn.linear_model import LassoLarsCV
regressor =  LassoLarsCV(normalize=True)
regressor.fit(X_train, y_train)

/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.978e-02, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. a

LassoLarsCV(copy_X=True, cv=None, eps=2.2204460492503131e-16,
      fit_intercept=True, max_iter=500, max_n_alphas=1000, n_jobs=1,
      normalize=True, positive=False, precompute='auto', verbose=False)

In [62]:
y_predict=regressor.predict(X_test)
print(r2_score( y_test,y_predict))

0.450505206125


9.GradientBoostingRegressor

In [65]:
from sklearn.ensemble import GradientBoostingRegressor
regressor=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.001, loss='huber', max_depth=3,
             max_features=0.55, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=18,
             min_samples_split=14, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=0.7, verbose=0, warm_start=False)

In [66]:
y_predict=regressor.predict(X_test)
print(r2_score( y_test,y_predict))

0.0588132353125


10,SGDRegressor

In [67]:
from sklearn import linear_model
regressor=linear_model.SGDRegressor()
regressor.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [68]:
y_predict=regressor.predict(X_test)
print(r2_score( y_test,y_predict))

-9.82240247916e+21


# Ranking List

In [ ]:
1.Ridge：linear_model.Ridge(alpha=31.99)                                         0.4518
2.LassoLarsCV：LassoLarsCV(normalize=True)                                       0.4505
3.ElasticNet：ElasticNet(alpha=0.001,l1_ratio=0.1)                               0.4498
4.AdaBoostRegressor：AdaBoostRegressor(n_estimators=30,learning_rate=0.1)        0.4498
5.svm.SVR(kernel='linear'):svm.SVR(kernel='linear',C=0.3)                        0.4258
6.Lasso:linear_model.Lasso(alpha=0.0001)                                         0.4247
7.RandomForest:RandomForestRegressor(n_estimators=150)                           0.4188
8.svm.SVR(kernel='rbf'):svm.SVR(kernel='rbf',C=2.0,epsilon=1.0)                  0.3287
9.GradientBoostingRegressor                                                      0.0588
  GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)
10.SGDRegressor:linear_model.SGDRegressor()                               -9.82240247916e+21